In [1]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
# from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_extra_data_embedding as training_testing_gru_extra_data_embedding

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")


os.environ["WANDB_NOTEBOOK_NAME"] = "GRU - v8.0.1 Topaz.ipynb"

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
torch.__version__

'2.1.1'

In [5]:
data_file = './data/topaz_data_w_bsp_new5_nz.fth'
df = pd.read_feather(data_file)

In [6]:
df.BSP

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
391881   NaN
391882   NaN
391883   NaN
391884   NaN
391885   NaN
Name: BSP, Length: 391886, dtype: float64

In [7]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
import rnn_tools.model_saver
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2018-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
# states = ["VIC"]
# states = ["NSW"]
states = ["NZ"]
# states = ['VIC']
extra_tracks = ["Wentworth Park", "Wagga", "Temora"]
only_tracks = ["Bendigo", "Geelong", "Traralgon", "Ballarat", "Shepparton", "Warragul","Healesville"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
# data_file = './data/topaz_data_w_bsp_new4_small.fth'
# data_file = './data/topaz_data_w_bsp_new4.fth'
data_file = './data/topaz_data_w_bsp_new6.fth'
   
raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,only_tracks=only_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='boosted_sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
(2518990, 156)
59
Latest date = 2024-05-19 00:00:00
size after state filter (2518990, 157)
(389662, 158)
Index(['index', 'meetingDate', 'state', 'track_name', 'dist', 'raceId',
       'race_grade', 'race_num', 'box', 'rugNumber',
       ...
       'prev_BSP_price', 'prev_start_price', 'stats_topaz', 'dogid', 'raceid',
       'stats_cols', 'track_hash', 'stats', 'race_time', 'stats_cuda'],
      dtype='object', length=159)
state  track_name  
NZ     Addington       97713
       Ascot Park      17255
       Ashburton         187
       Cambridge       25765
       Forbury Park     9035
       Hatrick         40965
       Manawatu        33986
       Manukau         29516
       Taranaki          351
       Tokoroa           282
Name: count, dtype: int64
Latest date = 2024-05-16 00:00:00
stats = [ 2.58278131e-01  1.00000000e+00  8.00000000e+00  5.04504502e-01
  0.00000000e+00  0.00000000e+00  1.95299149e-01  7.00000000e+00
  3.20000005e+00  5

100%|██████████| 25/25 [00:00<?, ?it/s]


race_ids=['285107145', '285107151', '285107150', '285107152', '285107149', '285107144', '285107143', '285107146', '285107153', '285107142', '285107148', '285107147', '284840710', '284840718', '284840712', '284840717', '284840715', '284840725', '284840721', '284840723', '284840727', '284840732', '284840731', '284840733', '284840729', '284840734', '284840730', '284840728', '284840726', '284840720', '284840724', '284840722', '284840713', '284840719', '284840716', '284840714', '284840709', '284942919', '284840711', '284942921', '284942924', '284942920', '284942922', '284942925', '284942923', '284942930', '284942928', '284942926', '284942927', '284942929', '284942873', '284942872', '284942874', '284942875', '284942878', '284942877', '284942876', '284942881', '284942879', '284942880', '286199760', '284942882', '284942883', '286199761', '286199762', '286199763', '286199766', '286199764', '286199765', '286199757', '286199759', '286199767', '286199768', '286199758', '286172802', '286172796', '2

In [ ]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def load_matching_state_dict(model, state_dict):
    model_dict = model.state_dict()

    # Filter out unnecessary keys
    matching_keys = {k: v for k, v in state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    excluded_keys = set(state_dict.keys()) - set(matching_keys.keys())

    # Overwrite entries in the existing state dict
    model_dict.update(matching_keys)

    # Load the new state dict
    model.load_state_dict(model_dict)

    print("Loaded keys:")
    for key in matching_keys:
        print(key)

    print("\nExcluded keys:")
    for key in excluded_keys:
        print(key)

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Reporting", config=config,save_code=True):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*6+reg_stat_mask+[0]*80
      # stat_mask = [1]*30
      stat_mask = [1]*27+53*[0]
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box,T_box, T_dist
      data_mask =  [1]*27+53*[1]
      # data_mask =  [1]*30
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      print(f"{data_mask=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-06-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra_embedding(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        load_matching_state_dict(model, model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        model = model.to(device)        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      try:
        return training_testing_gru_extra_data_embedding.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      raceDB.create_hidden_states_dict_v2()
      rnn_tools.model_saver.model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")


    return (model,dataset, optimizer)

In [ ]:
raceDB.batches_setup = False

In [ ]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':0,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':0,  
    '_pos_out_avg_1':0,
    '_post_change_avg_1':0,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':0,
    '_last_start_prob':0,
}

wandb_config_static = {
    'hidden_size': 128,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.0001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2022-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 450,
    'stat_list_dict': stat_list_dict,
}

In [ ]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])
torch.cuda.empty_cache()
raceDB.race_prices_to_prob()
print(raceDB.latest_date)
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  
    race.loss_log = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

for race in raceDB.racesDict.values():
    if race.classes.isnan().sum():
        print(race.classes.sum())
        print(race.raceid)
        race.classes = race.classes.nan_to_num(0,0,0)

2024-05-16 00:00:00
tensor(nan, device='cuda:0')
344127901
tensor(nan, device='cuda:0')
402888097
tensor(nan, device='cuda:0')
458549780
tensor(nan, device='cuda:0')
618363326


In [ ]:
torch.cuda.empty_cache()   

In [ ]:
x = torch.rand(10,8)
x.argmax(dim=-1)

tensor([6, 7, 2, 1, 1, 4, 4, 6, 0, 7])

In [ ]:
for r in raceDB.racesDict.values():
    try:
        r.start_prob
    except:
        print(r.raceid)
        print(r.raw_prices)

In [ ]:
for r in raceDB.racesDict.values():
    print(r.prices)
    # print(all([x==np.nan for x in r.prices]))
    # print(all([(np.isnan(x) or x==0) for x in r.prices]))
    print(r.fin_prob)
    print(r.start_prob)
    break

[nan, nan, nan, nan, nan, nan, nan, nan]
tensor([9.8857e-02, 5.3952e-02, 7.5413e-01, 2.6956e-02, 1.0000e-06, 5.1851e-02,
        7.3420e-03, 1.3321e-02], device='cuda:0')
tensor([9.8857e-02, 5.3952e-02, 7.5413e-01, 2.6956e-02, 1.0000e-06, 5.1851e-02,
        7.3420e-03, 1.3321e-02], device='cuda:0')


In [ ]:
for r in raceDB.racesDict.values():
    try:
        r.fin_prob
    except:
        print(r.raceid)
        print(r.raw_prices)
        # r

: 

In [18]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
importlib.reload(training_testing_gru_extra_data_embedding)
importlib.reload(rnn_tools.model_saver)
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NSW-lilac-sunset-824', prev_model_version = 60)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC SA NSW-ancient-valley-726', prev_model_version = 280) 


wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 128, 'stats': "['dogAgeScaled', 'boxNumber', 'incomingGrade_val', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_incomingGrade_val_mean_1', 'dog_startPrice_mean_1', 'dog_distance_mean_1', 'dog_boxNumber_mean_1', 'dog_runTimeNorm_mean_1', 'dog_win_runTimeNorm_mean_1', 'dog_delta_win_runTimeNorm_mean_1', 'dog_place_mean_1', 'dog_resultMargin_mean_1', 'dog_finishingPlaceMovement_mean_1', 'dog_StartingPlaceMovement_mean_1', 'dog_split_time_margin_mean_1', 'dog_split_runTimeNorm_mean_1', 'dog_time_1_mean_1', 'dog_run_home_TimeNorm_mean_1', 'dog_performance_to_price_mean_1', 'dog_averageSpeed_mean_1', 'dog_win_mean_1', 'dog_time_1_mean_365D', 'trainer_incomingGrade_val_mean_365D', 'trainer_startPrice_mean_365D', 'trainer_distance_mean_365D', 'trainer_boxNumber_mean_365D', 'trainer_runTimeNorm_mean_365D', 'trainer_win_runTimeNorm_mean_365D', 'trainer_delta_win_runTimeNorm_mean_365D', 'trainer_place_mean_365D', 'train

100%|██████████| 25861/25861 [00:17<00:00, 1486.80it/s]


[(datetime.date(2018, 1, 2), datetime.date(2019, 3, 28)), (datetime.date(2019, 3, 28), datetime.date(2020, 6, 20)), (datetime.date(2020, 6, 20), datetime.date(2021, 9, 13)), (datetime.date(2021, 9, 13), datetime.date(2021, 12, 31))]
2019-03-28
2020-06-20
2021-09-13
2021-12-31
Train examples [15659, 15012, 14843, 3956]


100%|██████████| 25860/25860 [00:09<00:00, 2602.39it/s]
4it [02:05, 31.29s/it]


...

 16%|█▌        | 400/2500 [6:59:31<32:57:29, 56.50s/it]

Quickpass time 11.713076100015314
Test val pass Time: 13.883199800009606
val val pass Time: 6.636249100003624
Test time : 44.03094970001257, Val time : 17.230458100006217, Total time : 61.26140780001879
New Max ROI: 0.0061, 0.056, 1.64869


 16%|█▌        | 406/2500 [7:06:45<37:03:37, 63.71s/it]

Quickpass time 11.738618300005328


 16%|█▋        | 407/2500 [7:07:37<34:59:53, 60.20s/it]

In [ ]:
wandb_config_static.update({'learning_rate':0.001})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.0005})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.00008})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.0001})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)
wandb_config_static.update({'learning_rate':0.005})
model_pipeline(raceDB,config=wandb_config_static,sweep=False)